# Load the dataframe
## README 
Website: https://www.eqtlgen.org/phase1.html

Paper: https://www.nature.com/articles/s41588-021-00913-zhttps://www.nature.com/articles/s41588-021-00913-z

This Jupyter notebook README covers cis-eQTL and trans-eQTL results from the eQTLGen project. The dataset includes significant files for both cis-eQTL and trans-eQTL analyses. 

Files contain various columns with information on P-value, SNP rs ID, SNP chromosome, SNP position, assessed and not assessed alleles, Z-score, ENSG and HGNC names of eQTL genes, gene chromosome, gene position, number of cohorts, number of samples, false discovery rate, and Bonferroni-corrected P-value.

The cis-eQTL analysis includes 19,250 genes expressed in blood, with SNP-gene combinations within 1Mb from the gene center and tested in at least 2 cohorts. The trans-eQTL analysis tests 19,960 genes expressed in blood and 10,317 trait-associated SNPs based on GWAS Catalog, Immunobase, and Astle et al. study. Trans-eQTL combinations have a distance of >5Mb and were tested in at least 2 cohorts.

The FDR calculation uses a pruned set of SNPs for trans-eQTL mapping and permutation-based FDR calculation. Crossmapping filters are applied to identify and remove potential artifacts in trans-eQTL results, recalculating the FDR afterward. Note that the full results file has not been filtered for cross-mapping effects, which may lead to artifacts in the data.

The code below demonstrates the process of creating a graph-based representation of the combined cis and trans-eQTL data using PyTorch Geometric. This process can be broken down into several steps:

1. Combine cis and trans dataframes: The code begins by concatenating the cis and trans dataframes into a single dataframe named 'data', which contains information on both cis-eQTL and trans-eQTL results. This combined dataset simplifies the process of working with the data and ensures that all relevant information is contained within a single data structure.

2. Create mappings for genes and SNPs: To represent the genes and SNPs as nodes in the graph, integer indices are assigned to each unique gene and SNP. This is done using dictionaries called 'gene_to_idx' and 'snp_to_idx', which map the gene and SNP identifiers to their corresponding integer indices.

3. Generate node type labels: Node type labels are created using PyTorch tensors, distinguishing between gene nodes (assigned a label of 0) and SNP nodes (assigned a label of 1). This differentiation is useful for various graph-based analyses and machine learning tasks that require knowledge of node types.

4. Create edges based on gene and SNP indices: Edges in the graph represent the relationships between genes and SNPs. These edges are created by iterating over the 'data' dataframe and extracting the corresponding gene and SNP indices from the previously created mappings. The edges are then represented as a PyTorch tensor with a long data type.

5. Convert edges to undirected: Since the relationships between genes and SNPs are undirected, the edges in the graph should also be undirected. This is achieved using the 'to_undirected()' function from PyTorch Geometric, which ensures that the graph correctly represents the underlying biology.

6. Create a PyTorch Geometric graph: Finally, the graph is created using the PyTorch Geometric 'Data' class. The node types and edge indices are used as inputs to instantiate the graph object, which can then be utilized for further analysis and visualization.

The resulting 'graph' object is a PyTorch Geometric representation of the combined cis and trans-eQTL data. The prediction task is to predict new association edges given the training edges, with the task type being link prediction. Below are a few important graph statistics:

- Number of nodes: 3681495
- Number of SNP nodes: 3664025
- Number of Gene nodes: 17470
- Number of edges: 10567450
- Number of connected components: 424
- Average degree: 5.74
- Median degree: 2.0
- Standard deviation of degree: 69.81
- Density: 0.0000015594
- Assortativity: -0.2267915607

The prediction task is to predict new association edges given the training edges, with the task type being link prediction. The data splitting is random while maintaining an equal proportion of cis- and trans- associations.

## Data Setup 

### Libraries

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import to_undirected, negative_sampling, subgraph

import networkx as nx
from ogb.io import DatasetSaver
from ogb.linkproppred import LinkPropPredDataset

In [2]:
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Geometric version: {torch_geometric.__version__}")

PyTorch version: 2.0.0+cu118
PyTorch Geometric version: 2.3.1


### Load data

In [6]:
# Read files
data_types = {'Pvalue': float, 'SNP': str, 'SNPChr': str, 'SNPPos': float,
              'AssessedAllele': str, 'OtherAllele': str, 'Zscore': float,
              'Gene': str, 'GeneSymbol': str, 'GeneChr': str, 'GenePos': float,
              'NrCohorts': int, 'NrSamples': int, 'FDR': float,
              'BonferroniP': float, 'GeneStart': float, 'GeneEnd': float}
data = pd.read_csv("sig-combined-with-genes.csv", dtype=data_types)

In [7]:
# Create mappings for genes and SNPs to integer indices
genes = data['Gene'].unique()
snps = data['SNP'].unique()
gene_to_idx = {gene: idx for idx, gene in enumerate(genes)}
snp_to_idx = {snp: idx + len(genes) for idx, snp in enumerate(snps)}

# Create node feature vectors
gene_features = data.loc[data['Gene'].isin(genes)][['Gene', 'GeneChr', 'GeneStart', 'GeneEnd']].drop_duplicates().sort_values(by='Gene').reset_index(drop=True)
snp_features = data.loc[data['SNP'].isin(snps)][['SNP', 'SNPChr', 'SNPPos', 'AssessedAllele', 'OtherAllele']].drop_duplicates().sort_values(by='SNP').reset_index(drop=True)

# Create node type labels
node_types = torch.tensor([0] * len(genes) + [1] * len(snps), dtype=torch.long)

# Create edges
edges = data.apply(lambda row: (gene_to_idx[row['Gene']], snp_to_idx[row['SNP']]), axis=1)
edges = torch.tensor(list(edges), dtype=torch.long).t().contiguous()

# Convert edges to undirected
edges = to_undirected(edges)

# Combine the feature vectors
combined_features = pd.concat([gene_features, snp_features], ignore_index=True).drop(['Gene', 'SNP'], axis=1)

# Convert 'GeneChr' and 'SNPChr' to strings
combined_features['GeneChr'] = combined_features['GeneChr'].astype(str)
combined_features['SNPChr'] = combined_features['SNPChr'].astype(str)

# Label encoding for categorical columns
categorical_columns = ['GeneChr', 'SNPChr', 'AssessedAllele', 'OtherAllele']
for column in categorical_columns:
    le = LabelEncoder()
    combined_features[column] = le.fit_transform(combined_features[column])

# Create the PyTorch tensor
features = torch.tensor(combined_features.values, dtype=torch.float)

# Create the PyTorch Geometric graph
graph = Data(x=features, edge_index=edges)

print(graph)

Data(x=[3681584, 7], edge_index=[2, 21134900])


In [8]:
def create_networkx_graph(edge_index):
    G = nx.Graph()
    for edge in edge_index.t().numpy():
        G.add_edge(edge[0], edge[1])
    return G

# Convert the PyTorch Geometric graph to a NetworkX graph
G = create_networkx_graph(graph.edge_index)

# Find the connected components
components = list(nx.connected_components(G))

# Find the largest component
largest_component = max(components, key=len)

# Get the node indices of the largest connected component
largest_component_nodes = torch.tensor(list(largest_component), dtype=torch.long)

# Extract the largest connected component subgraph
largest_subgraph_edge_index, _ = subgraph(largest_component_nodes, graph.edge_index)
largest_subgraph_features = graph.x[largest_component_nodes]

# Create the new Data object for the largest subgraph
largest_subgraph = Data(x=largest_subgraph_features, edge_index=largest_subgraph_edge_index)

print(largest_subgraph)

Data(x=[3576300, 7], edge_index=[2, 20865032])


In [9]:
def print_graph_stats(graph, nodes, genes, snps):
    G = nx.Graph()
    for edge in graph.edge_index.t().numpy():
        G.add_edge(edge[0], edge[1])

    num_nodes = G.number_of_nodes()
    num_genes = len([node for node in nodes if node < len(genes)])
    num_snps = len([node for node in nodes if node >= len(genes)])
    num_edges = G.number_of_edges()
    num_connected_components = nx.number_connected_components(G)
    average_degree = np.mean([degree for _, degree in G.degree()])
    median_degree = np.median([degree for _, degree in G.degree()])
    std_degree = np.std([degree for _, degree in G.degree()])
    density = nx.density(G)
    assortativity = nx.degree_assortativity_coefficient(G)

    print(f"Number of nodes: {num_nodes}")
    print("Number of SNP nodes:", num_snps)
    print("Number of Gene nodes:", num_genes)
    print(f"Number of edges: {num_edges}")
    print(f"Number of connected components: {num_connected_components}")
    print(f"Average degree: {average_degree:.2f}")
    print(f"Median degree: {median_degree}")
    print(f"Standard deviation of degree: {std_degree:.2f}")
    print(f"Density: {density:.10f}")
    print(f"Assortativity: {assortativity:.10f}")

# For the full graph
print("Full graph stats:")
print_graph_stats(graph, range(graph.num_nodes), genes, snps)
print("\n")

# For the subgraph
print("Subgraph stats:")
print_graph_stats(largest_subgraph, largest_component_nodes.numpy(), genes, snps)

Full graph stats:
Number of nodes: 3681495
Number of SNP nodes: 3664114
Number of Gene nodes: 17470
Number of edges: 10567450
Number of connected components: 424
Average degree: 5.74
Median degree: 2.0
Standard deviation of degree: 69.81
Density: 0.0000015594
Assortativity: -0.2267915607


Subgraph stats:
Number of nodes: 3576300
Number of SNP nodes: 3559409
Number of Gene nodes: 16891
Number of edges: 10432516
Number of connected components: 1
Average degree: 5.83
Median degree: 2.0
Standard deviation of degree: 70.62
Density: 0.0000016314
Assortativity: -0.2275165079


In [13]:
def custom_inverse_transform(le, indices):
    decoded = np.empty_like(indices, dtype=object)
    for i, idx in enumerate(indices):
        if 0 <= idx < len(le.classes_):
            decoded[i] = le.classes_[idx]
        else:
            decoded[i] = np.nan
    return decoded

# Extract the node features from the largest subgraph
node_features = largest_subgraph.x.numpy()

# Decode the categorical columns
categorical_columns = ['GeneChr', 'SNPChr', 'AssessedAllele', 'OtherAllele']
continuous_columns = ['GeneStart', 'GeneEnd', 'SNPPos']
encoders = {}

# Retain continuous columns as they are
continuous_features = node_features[:, [1, 2, 4]]

# Process categorical columns
categorical_features = []
for i, column in enumerate(categorical_columns):
    le = LabelEncoder()
    le.fit(combined_features[column])
    categorical_feature_idx = np.round(node_features[:, i]).astype(int)
    categorical_features.append(custom_inverse_transform(le, categorical_feature_idx))
    encoders[column] = le

# Combine continuous and categorical columns
processed_node_features = np.column_stack((categorical_features[0], continuous_features[:, 0], continuous_features[:, 1],
                                           categorical_features[1], continuous_features[:, 2], categorical_features[2], categorical_features[3]))

# Create column names for the node features
column_names = ['GeneChr', 'GeneStart', 'GeneEnd', 'SNPChr', 'SNPPos', 'AssessedAllele', 'OtherAllele']

# Create a new DataFrame with the original column names and values
largest_subgraph_df = pd.DataFrame(processed_node_features, columns=column_names)

# Reset the index of the DataFrame
largest_subgraph_df.reset_index(drop=True, inplace=True)

C:\Users\falty\AppData\Local\Temp\ipykernel_4204\2184106801.py:26: RuntimeWarning: invalid value encountered in cast
  categorical_feature_idx = np.round(node_features[:, i]).astype(int)
C:\Users\falty\AppData\Local\Temp\ipykernel_4204\2184106801.py:26: RuntimeWarning: invalid value encountered in cast
  categorical_feature_idx = np.round(node_features[:, i]).astype(int)


In [11]:
largest_subgraph_df.head()

,GeneChr,GeneStart,GeneEnd,SNPChr,SNPPos,AssessedAllele,OtherAllele
0,12,50934868.0,50959140.0,NaN,NaN,NaN,NaN
1,0,169849632.0,169894272.0,NaN,NaN,NaN,NaN
2,0,169662000.0,169854080.0,NaN,NaN,NaN,NaN
3,0,27612064.0,27635184.0,NaN,NaN,NaN,NaN
4,0,196651760.0,196752480.0,NaN,NaN,NaN,NaN
